# AI-LAB SESSION 2: Informed search

In the second session we will work on informed search

## Maze environments

The environments used are again **SmallMaze** (visible in the figure) and its variations
![SmallMaze](images/maze.png)
The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$

## Priority Fringe

You will need a queue ordered by priority as fringe, or **PriorityFringe**. The difference between the other versions of fringe is that in **PriorityFringe** nodes are removed from the fringe based on the current lowest value. In particular, **FringeNode** has two useful parameters (other than those used in the previous session):
* *pathcost* - the path cost from the root node to the current one (defaults to 0)
* *value* - value of a node. Used by **PriorityFringe** to order its content (defaults to 0)

Here is an example of usage

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from timeit import default_timer as timer
from utils.fringe import FringeNode, PriorityFringe

# Create 3 nodes for state ids 1 2 3
node_1 = FringeNode(1) # No parent, pathcost=0, value=0
node_2 = FringeNode(2, node_1, node_1.pathcost + 1, 10) # Child of node_1, value=10
node_3 = FringeNode(3, node_1, 100, 5)  # Child of node_1, pathcost=100, value=5

p_fringe = PriorityFringe()
for n in (node_1, node_2, node_3):
    p_fringe.add(n)
    print("Added: {}".format(n.state))

while not p_fringe.is_empty():
    print("Removed: {}".format(p_fringe.remove().state))

Added: 1
Added: 2
Added: 3
Removed: 1
Removed: 3
Removed: 2


Notice the order in which nodes are removed from the fringe

## Uniform-Cost Search (UCS)

Before moving to informed search it is useful to see another uninformed search algorithm: the Uniform-Cost Search (UCS). In the following you can see the implementation in *tree search* version. Cost of performing an action is supposd to be 1 (also in the assignements).

In [2]:
import gym
import envs
from utils.funcs import build_path


def ucs(environment):
    """
    Uniform-cost search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        path: solution as a path
    """
    fringe = PriorityFringe()
    fringe.add(FringeNode(environment.startstate))
    while True:
        if fringe.is_empty():
            return None
        node = fringe.remove()  # Retrieve node from the fringe
        if node.state == environment.goalstate:  # Goal state check
            return build_path(node)
        for action in range(environment.action_space.n):  # Look around
            # Child node where value and pathcost are both the pathcost of parent + 1
            child = FringeNode(environment.sample(node.state, action), node, node.pathcost + 1, node.pathcost + 1)
            fringe.add(child)

# Create and render the environment
env = gym.make("SmallMaze-v0")
env.render()
solution = ucs(env)

# Print path
print("\nSolution: {}".format([env.state_to_pos(s) for s in solution]))

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]

Solution: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


## Distance Heuristics

Informed search requires a distance heuristic to be used in order to estimate the distance between a state and the goal. You already have at your disposal these functions:
* *l1_norm(p1, p2)* - Computes the L1 norm (also known as the manhattan distance) between two points specified as tuples of coordinates
* *l2_norm(p1, p2)* - Computes the L2 norm between two points specified as tuples of coordinates
* *chebyshev(p1, p2)* - Computes the Chebyshev distance between two points specified as tuples of coordinates. Similar to L1 norm but diagonal moves are also considered

Examples:

In [3]:
import utils.heuristics as heu

p1 = (0, 2)
p2 = (4, 0)
print("L1 norm heuristic value: {}".format(heu.l1_norm(p1, p2)))
print("L2 norm heuristic value: {}".format(heu.l2_norm(p1, p2)))
print("Chebyshev heuristic value: {}".format(heu.chebyshev(p1, p2)))

L1 norm heuristic value: 6
L2 norm heuristic value: 4.47213595499958
Chebyshev heuristic value: 4


## Assignment 1

The first assignment is to implement the Greedy-best-first search algorithm on **SmallMaze**. In particular, you are required to implement both *greedy_tree_search* and *greedy_graph_search* versions that will be called by the generic *greedy*. Use *L1 norm* as heuristic function at first, then try also the others to see the differences.

The results returned by *greedy* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (fringe + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

The next two functions have to be implemented

In [13]:
def greedy_tree_search(env):
    """
    Greedy-best-first Tree search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    fringe = PriorityFringe() #mette davanti i valori più bassi
    fringe.add(FringeNode(env.startstate))
    i = 1
    while True:
        if fringe.is_empty():
            return None, (0, 0)
        node = fringe.remove()
        if node.state == env.goalstate:
            return build_path(node),(i,len(fringe) + 1)
        else:
            for action in range(env.action_space.n):
                next_state = env.sample(node.state, action)
                child = FringeNode(next_state, node, node.pathcost + 1, heu.l1_norm(env.state_to_pos(next_state), env.state_to_pos(env.goalstate)))
                fringe.add(child)  
                #child = FringeNode(env.sample(node.state, action), node,heu.l1_norm(env.state_to_pos(node.state), env.state_to_pos(env.sample(node.state, action))))
                #child = FringeNode(next_state, node, node.pathcost + 1, heu.l1_norm(env.state_to_pos(next_state), env.state_to_pos(goal)))
                #fringe.add(child)
        i+=1

In [14]:
def greedy_graph_search(environment):
    """
    Greedy-best-first Graph search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    
    i = 0
    mem = 0
    
    goal = environment.goalstate
    
    fringe = PriorityFringe()
    fringe.add(FringeNode(environment.startstate))
    
    closed = []
    while True:
        if fringe.is_empty():
            return None, (i,mem)
        
        node = fringe.remove()   
        if node.state == environment.goalstate:
            return build_path(node), (i+1,mem)
        i+=1
        if node.state not in closed :  
            closed.append(node.state)
            for action in range(environment.action_space.n):  
                next_state = environment.sample(node.state, action)
                child = FringeNode(next_state, node, node.pathcost + 1, heu.l1_norm(environment.state_to_pos(next_state), environment.state_to_pos(env.goalstate)))
                fringe.add(child)        
            if mem < (len(fringe)+len(closed)):
                mem = (len(fringe)+len(closed))
    
    return None, (i, mem)

In [15]:
def greedy(environment, search_type):
    """
    Greedy-best-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - greedy_tree_search or greedy_graph_search (function pointer)
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, stats = search_type(environment)
    return path, (timer() - t, stats[0], stats[1])

The following code calls your tree search version of Greedy-best-first search and prints the results

In [19]:
envname = "BlockedMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")
#CORRETTO##
# Create and render the environment
env = gym.make(envname)

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = greedy(env, greedy_graph_search)  # Perform Greedy-best-first search
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


Execution time: 0.0062s
N° of nodes explored: 61
Max n° of nodes in memory: 24
Solution: None


In [17]:
envname = "BlockedMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

##CORRETTO###

# Create and render the environment
env = gym.make(envname)

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = greedy(env, greedy_tree_search)  # Perform Greedy-best-first search
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


KeyboardInterrupt: 

Correct results for Greedy-best-first tree search can be found [here](results/greedy_tree_search_results.txt)

The following code calls your graph search version of Greedy-best-first search and prints the results

Correct results for Greedy-best-first graph search can be found [here](results/greedy_graph_search_results.txt)

## Assignment 2

The second assignment is to implement the A* search algorithm on **SmallMaze**. In particular, you are required to implement both *astar_tree_search* and *astar_graph_search* versions that will be called by the generic *astar*. Use *L1 norm* as heuristic function at first, then try also the others to see the differences.

The results returned by *astar* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (fringe + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

The next two functions have to be implemented

In [12]:
def astar_tree_search(environment):
    """
    A* Tree search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    fringe = PriorityFringe() #mette davanti i valori più bassi
    fringe.add(FringeNode(env.startstate))
    i = 1
    while True:
        if fringe.is_empty():
            return None, (0, 0)
        node = fringe.remove()
        if node.state == env.goalstate:
            return build_path(node),(i,len(fringe) + 1)
        for action in range(env.action_space.n):
            child = FringeNode(env.sample(node.state, action), node,node.pathcost + 1, heu.l1_norm(env.state_to_pos(env.sample(node.state, action)), env.state_to_pos(env.goalstate)) + node.pathcost + 1)
            fringe.add(child)
        i+=1

In [21]:
def astar_graph_search(environment):
    """
    A* Graph search
    
    Args:
        environment: OpenAI Gym environment
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    i = 0
    m_node = 0
    closed = []
    
    goal = environment.goalstate
    
    fringe = PriorityFringe()
    fringe.add(FringeNode(environment.startstate))
    while True:
        if fringe.is_empty():
            return None, (i,m_node)
        node = fringe.remove()
        i+=1
        if node.state == environment.goalstate : 
            return build_path(node), (i,m_node)
        if node.state not in closed :
            closed.append(node.state)
            node.pathcost += 1
            for action in range(environment.action_space.n): 
                next_state = environment.sample(node.state, action)
                h = heu.l1_norm(environment.state_to_pos(next_state),environment.state_to_pos(goal))
                child = FringeNode(next_state, node, node.pathcost, h + node.pathcost)
                fringe.add(child)
            #m_node = max(m_node, len(closed))
            if m_node < (len(fringe) + len(closed)):
                m_node = len(fringe) + len(closed)

    return None, (i, m_node)

In [14]:
def astar(environment, search_type):
    """
    A* search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - astar_tree_search or astar_graph_search (function pointer)
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, stats = search_type(environment)
    return path, (timer() - t, stats[0], stats[1])

The following code calls your tree search version of A* search and prints the results

In [17]:
envname = "BlockedMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")
##CORRETTO###
# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = astar(env, astar_tree_search)  # Perform A* search
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  BlockedMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'C' 'W' 'W']
 ['C' 'C' 'W' 'G']]


KeyboardInterrupt: 

Correct results for A* tree search can be found [here](results/astar_tree_search_results.txt)

The following code calls your graph search version of A* search and prints the results

In [24]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

##CORRETTO###

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = astar(env, astar_graph_search)  # Perform A* search
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 0.0062s
N° of nodes explored: 43
Max n° of nodes in memory: 34
Solution: [(0, 1), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for A* graph search can be found [here](results/astar_graph_search_results.txt)

## Discussion

Now that you have correctly implemented both Greedy-best-first and A* what can you say about the solutions they compute? Are there significant differences in the stats? Try to play with other heuristics as well and see if your results change.